In [ ]:
from sympy import *

def objective_function_user():
    varibs = input("Type in your variable names(seperated with single spaces): ")
    v=list(filter(None,varibs))
    
    varibs = symbols(varibs)
    
    method_list = ['acos', 'asin', 'atan', 'atan2', 'ceil', 'cos', 
                 'cosh', 'degrees', 'e', 'exp', 'fabs', 'floor', 
                 'fmod', 'frexp', 'hypot', 'ldexp', 'log', 'log10', 
                 'modf', 'pi', 'pow', 'radians', 'sin', 'sinh', 'sqrt', 
                 'tan', 'tanh']

    while(True):
        opt_type = input('Enter optimization type "Max or Min": ')
        if (opt_type=='Min')|(opt_type=='Max'):
            break;
        else:
            print("Incorrect input; Please type Min or Max")
            continue
    obj_user = input('`````\n\
    Note for objective function\n\n\
    Use format as shown in the following example:\n\n\
    2*x**2+log(x)+sin(x+sqrt(x))+exp(x**3)-acos(x)\n\n\
    "**"(double asterisk means power or exponent i.e. x**2= x squared)\n\n\
    Always use \'*\' to denote multiplication\n\n\
    Please only use variables that you have enetered before\n\n\
    list of standard funtions --> \n\n\
    %s\
    \n\nEnter your objective function: '%method_list)
    if type(varibs)!=tuple:
        num_vars=1
        count = obj_user.count(str(varibs))
        while count!=0:
            temp='varibs'
            obj_user=obj_user[0:obj_user.index(str(varibs))]+temp+obj_user[obj_user.index(str(varibs))+len(str(varibs)):]
            count=count-1
    else:
        num_vars=len(varibs)
        for j,j2 in zip(list(varibs),range(num_vars+1)):
            count = obj_user.count(str(j))
            while count!=0:
                temp='varibs[%i]'%(j2)
                obj_user=obj_user[0:obj_user.index(str(j))]+temp+obj_user[obj_user.index(str(j))+len(str(j)):] 
                count=count-1
    print(obj_user)
    obj = eval(obj_user)
    
    return obj,safe_dict,opt_type,num_vars,varibs



def exhaustive_search_opt():
    print("Please use only ONE VARIABLE for this method")
    user_inputs = objective_function_user()
    obj_exhaustive = user_inputs[0]#eval('sin(x)')#2*x**2-x**3
    opt_type = user_inputs[2]
    num_vars = user_inputs[3]
    varibs = user_inputs[4]
    if num_vars>1:
        print("\n\nCannot perform multivariate optimization using this method")
        return 0
    step_size = int(input('Enter number of parts of the function: '))
    # constraint 0.0<=x<=2.0
    x_min = float(input('Enter a minimum value for the variable range: '))
    x_max = float(input('Enter a maximum value for the variable range: '))
    del_x = (x_max-x_min)/step_size
    x_curr = x_min

    pl=plot(obj_exhaustive,(varibs,x_min,x_max),show=false)
    pl.show()
    iter_var=0
    while (x_curr<=x_max):
        x_1 = x_curr
        x_2 = x_1+del_x
        x_3 = x_2+del_x
        f_x_1=obj_exhaustive.subs({varibs:x_1})
        f_x_2=obj_exhaustive.subs({varibs:x_2})
        f_x_3=obj_exhaustive.subs({varibs:x_3})
        iter_var = iter_var+1
        if opt_type=='Max':
            if ((f_x_1<f_x_2)&(f_x_3<f_x_2)):
                print('\n\nApprox. optimum value may be: %f; found at: %s =  %f; in %i iterations '% (f_x_2,varibs,x_2,iter_var))
                break
            else:
                x_curr = x_2
                continue
        elif opt_type=='Min':
            if ((f_x_1>f_x_2)&(f_x_3>f_x_2)):
                print('\n\nApprox. optimum value may be: %f; found at: %s =  %f; in %i iterations '% (f_x_2,varibs,x_2,iter_var))
                break
            else:
                x_curr = x_2
                continue

def random_walk_opt():
    user_inputs = objective_function_user()
    obj_rw = user_inputs[0]#eval('sin(x)')#2*x**2-x**3
    opt_type = user_inputs[2]
    num_vars = user_inputs[3]
    print(num_vars)
    varibs = user_inputs[4]
    constraints = []
    initial_solution = np.zeros(num_vars)
    if num_vars==1:
        temp1=float(input('Enter minimum limit of %s: '%varibs))
        while True:
            temp2=float(input('Enter maximum limit of %s: '%varibs))
            if temp2>temp1:
                break
            else:
                print('You must enter a value grater than %f'%temp2)
                continue
    else:    
        for ctr in range(num_vars):
            temp1=float(input('Enter minimum limit of %s: '%varibs[ctr]))
            while True:
                temp2=float(input('Enter maximum limit of %s: '%varibs[ctr]))
                if temp2>temp1:
                    break
                else:
                    print('You must enter a value grater than %f'%temp2)
                    continue
            #print(temp1,temp2)
        constraints.append([temp1,temp2])
        print('Generating random initial solution....................')
        initial_solution[ctr]=np.random.uniform(temp1,temp2)
    value_dict = dict(zip(list(i for i in varibs),initial_solution))
    lamb = float(input('Enter lambda value: '))
    epsilon = float(input('Enter epsilon value: '))
    n_iters = int(input('Enter number of iterations: '))
    function_value = initial_func_val = obj_rw.subs(value_dict)
    varib_vals = np.resize(initial_solution,(num_vars,1))
    run=3
    for iteration in range(n_iters):
        random_vals = np.random.uniform(-1.0,1.0,size=(num_vars,1))
        rms_random_vals = np.sum(np.square(random_vals))
        if rms_random_vals>=1.0:
            continue
            
        unit_random_vector = np.divide(random_vals,rms_random_vals)
        
        new_varib_vals = np.add(varib_vals,np.multiply(lamb,unit_random_vector))
        
        value_dict = dict(zip(list(i for i in varibs),np.resize(new_varib_vals,num_vars)))
        new_func_value = obj_rw.subs(value_dict)

        if ((opt_type=='Max')&(new_func_value<function_value))|((opt_type=='Min')&(new_func_value>function_value)):
            continue

        function_value=new_func_value
        varib_vals=new_varib_vals
        if (iteration==n_iters-1)&(((opt_type=='Max')&(initial_func_val>function_value))|((opt_type=='Min')&(new_func_value>function_value)))&(run!=0):
            run=run-1
            iteration=0
            lamb=0.5*lamb
    
        if (iteration==n_iters-1)&(run==0)&((opt_type=='Max')&(new_func_value<function_value))|((opt_type=='Min')&(new_func_value>function_value)):
            print('A feasible solution could not be found')
        elif (iteration==n_iters-1)|(run==0)&((opt_type=='Max')&(new_func_value>function_value))|((opt_type=='Min')&(new_func_value<function_value)):   
            print('Approx optimal solution is: %f;\n found at %s;\n in %i iterations;\n after %i run(s)'%(function_value,value_dict,n_iters,run))
        print(iteration,run)

In [ ]:
exhaustive_search_opt()
random_walk_opt()